In [132]:
import tensorflow as tf
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# why dont these python scripts auto save in the directory..? 
    # Do I need to sync them or something? 
    

In [136]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

 
import datetime
from scipy.io import loadmat
import scipy
from scipy import stats


#4 covariate model with rain, n20, wfps, [and n2osf]
    # SL = 15, 30, 60  day sequences 
    # n2osf = 1, 3, 5, 7, 15 day sequences 
    # DO = 0.2,0.3,0.4   (0.5?)
    
# set some [LSTM model] variables 
SL="30"
n2osf="05"
DO="20"

n2osfI= int(n2osf)
SLI= int(SL)
DOI= int(DO)/100 

filenameX='/n2o_nn/LSTM_June2022/2_matlab_traintest_matdata/XTrain_base_SL'+SL+"_n2osf"+n2osf+".mat"
filenameY='/n2o_nn/LSTM_June2022/2_matlab_traintest_matdata/YTrain_base_SL'+SL+"_n2osf"+n2osf+".mat"
filenameW='/n2o_nn/LSTM_June2022/2_matlab_traintest_matdata/Weights_base_SL'+SL+"_n2osf"+n2osf+".mat"
print(filenameX)

# load the mat data files     
matx = loadmat(filenameX) 
maty = loadmat(filenameY)
matw = loadmat(filenameW)

# hdfname='/n2o_nn/LSTM_June2022/4_Python_LSTMmodels_hdf5/base_SL'+SL+"_n2osf"+n2osf+"_DO"+DO+"_w_not_norm.hdf5"
hdfname='\Users\cddorich\4_Python_LSTMmodels_hdf5base_SL'+SL+"_n2osf"+n2osf+"_DO"+DO+"_w_not_norm.hdf5'

print(hdfname)
# potential other covariates that could be added:
# learningrate = lr001
# epoch = epoch500
# rmsprop = rmsprop



/n2o_nn/LSTM_June2022/2_matlab_traintest_matdata/XTrain_base_SL30_n2osf05.mat
/n2o_nn/LSTM_June2022/4_Python_LSTMmodels_hdf5/base_SL30_n2osf05_DO20_w_not_norm.hdf5


In [137]:
print(tf.__version__)

1.15.0


In [138]:
# data prep for LSTM 

matx.keys()
maty.keys()
matx
dfx = pd.DataFrame(np.hstack((matx['XTrain1'])))
dfx.head
dfy = pd.DataFrame(np.hstack((maty['YTrain1'])))
dfy.head
dfw = pd.DataFrame(np.hstack((matw['Weights'])))
dfw.head

arrx = dfx.values
arry = dfy.values
arrw = dfw.values
resultx = arrx[:,0]
resulty = arry[:,0]
resultw = arrw[:,0]
 
xlist = []
ylist  = []
wlist  = []

for i in range(len(resultx)):
    xlist.append(np.transpose(resultx[i]))
    ylist.append(np.transpose(resulty[i]))
    
Xtrain = np.asarray(xlist)
Xtrain 

#print( len(Xtrain) )
#print( len(xlist) )

Xtrain = np.asarray(xlist)

nvals=len(xlist)*SLI
print(nvals)
ynvals=len(ylist)*SLI

# this value is the rows/cells of the Xtrain * the SL, and # of covariates
Xtrain = np.reshape(Xtrain, (nvals, 4)) 
dfx = pd.DataFrame(Xtrain)

Ytrain = np.asarray(ylist).reshape(ynvals,1) 
dfy = pd.DataFrame(Ytrain)

dfx.columns = ['sampn2o', 'rainseq','wfpsseq','n2o']
df_targets = pd.DataFrame(dfy)
df_targets.columns = ['target n2o']

x_data = dfx.values
y_data = df_targets.values
num_data = len(x_data)
num_data 

#new training and testing for 30 day sequences and with lesser hidden neurons. Comment it out to go back to the original 60 day sequence batches
scaler = StandardScaler()
print(scaler.fit(y_data))
print(scaler.mean_)
y_data =scaler.transform(y_data)
y_data.shape

x_data = x_data.reshape((len(xlist),SLI,4))  #30 day 7 cov repeat after mistake , day since and fert applied + soil temp 10 with new hold out 
y_data = y_data.reshape((len(ylist),SLI,1))

x_data.shape, y_data.shape
 
    
#try standarsizing the target variable as the model weights were exploding due to the large variation in the y  or the target data


72180
StandardScaler()
[11.56079198]


((2406, 30, 4), (2406, 30, 1))

In [139]:
len(xlist)

2406

In [140]:
#AL 6-14-22 sample_weights
wlist=resultw.astype(int)
sample_weight=[]

for i in wlist:
    sample_weight+=[1/i]*i

sample_weight=np.array(sample_weight)
sample_weight.shape
# sample_weight/=np.mean(sample_weight)

(2406,)

In [141]:
sample_weight[:1000]

array([0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 ,
       0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 ,
       0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 ,
       0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 ,
       0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 ,
       0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 ,
       0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 ,
       0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 ,
       0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 , 0.0212766 ,
       0.0212766 , 0.0212766 , 0.05555556, 0.05555556, 0.05555556,
       0.05555556, 0.05555556, 0.05555556, 0.05555556, 0.05555556,
       0.05555556, 0.05555556, 0.05555556, 0.05555556, 0.05555556,
       0.05555556, 0.05555556, 0.05555556, 0.05555556, 0.05555556,
       0.01785714, 0.01785714, 0.01785714, 0.01785714, 0.01785714,
       0.01785714, 0.01785714, 0.01785714, 0.01785714, 0.01785

In [142]:
# #faster more complicated array version, not working
# wlist=resultw.astype(int)
# sample_weight=np.zeros(x_data.shape[0])
# index=0
# for i in wlist:
#     sample_weight[index]=[1/i]*i
#     index+=1

# sample_weight=np.array(sample_weight)
# sample_weight

In [143]:
len(xlist)

2406

In [144]:
#LSTM architecture here  
    
# notes/architecture to QC 
    # changing the architecture to reduce hidden layer neurons to prevent overfitting with 25 neurons,
    # 50 precent dropout
    #and 30 time steps as input sequences. 
    
from keras.layers import Dropout

# from tensorflow.keras import layers 

model = Sequential()
# model = keras.Sequential() # tf2.3

# is this needed...? 
# # Add an Embedding layer expecting input vocab of size 1000, and
# # output embedding dimension of size 64.
# model.add(layers.Embedding(input_dim=1000, output_dim=64))

# for tensorflow 2.3
# https://www.tensorflow.org/guide/keras/rnn
# model.add(layers.LSTM(128,activation='tanh',
#                return_sequences=True,
#                ) )
# need to figure out the input_shape parameter 

# print(model)

# need to see about tensorflow syntax, AN used 1.15
#model.add(LSTM(50,activation='tanh',
#              return_sequences=True,  
#             input_shape=(30,7)))

model.add(LSTM(50,                  # what is this 50? layers I believe...
             activation='tanh',      # activation function 
             return_sequences=True,   
             input_shape=(SLI,4)))    # SL and covariates 

# this DO part needs to be vetted/tested ## 
  # if doing dropout 0.4 then need both 50 and 32 neuron 
    # otherwise dropout 0.2, 0.3 use just 50 layer 
  
    # if DOI=0.4, then add this second layer code line:
# model.add(LSTM(32, activation='tanh', return_sequences=True))

model.add(Dropout(DOI))    # set dropout layers 

##model.add(Dense(num_y_signals, activation='linear'))
model.add(Dense(1))
# # model.add(Dense(1, activation='relu'))
 
 
 # dont think below is needed...?:
# if False:
#     from tensorflow.python.keras.initializers import RandomUniform

#     # Maybe use lower init-ranges.
#     init = RandomUniform(minval=-0.05, maxval=0.05)

#     model.add(Dense(num_y_signals,
#                     activation='linear',
#                     kernel_initializer=init))
 
 

In [145]:
# optimizer = adam(lr=1e-2)
# optimizer
# from keras.optimizers import adam
from keras.optimizers import SGD
from keras.optimizers import RMSprop
# opt = adam(lr=0.001)

opt = RMSprop(lr=0.001)


In [146]:
model.compile(loss='mse', optimizer=opt)

In [147]:
# model.build()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 30, 50)            11000     
_________________________________________________________________
dropout_5 (Dropout)          (None, 30, 50)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 30, 1)             51        
Total params: 11,051
Trainable params: 11,051
Non-trainable params: 0
_________________________________________________________________


In [148]:
from keras.callbacks import ModelCheckpoint
path_checkpoint = '23_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [149]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=20, verbose=1) #initially patience was 5 for the 60 sequence model, now let's make it 20 to give the model more attempts at convergence 

In [150]:
callback_tensorboard = TensorBoard(log_dir='./23_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [151]:
 callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=20,
                                       verbose=1)

In [152]:
# callbacks = [callback_early_stopping,
#              callback_checkpoint,
#              callback_tensorboard,
#              callback_reduce_lr]

x_data.shape
y_data.shape

 
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min', restore_best_weights=True)

# mcp_save = ModelCheckpoint(filepath = '/Users/abhijnannath/crossvalid_18_30day_mdlwts_4covariate_base_Do20_lr001_epoch500_rmsprop_new_monitorloss_aug5.hdf5', save_best_only=True, monitor='loss', mode='min')
mcp_save = ModelCheckpoint(filepath = hdfname, save_best_only=True, monitor='loss', mode='min')
# mcp_save = ModelCheckpoint(filepath = '/n2o_nn/30day_7covariate_Do20_lr001_epoch500_rmsprop_new_monitorloss_n2osf7.hdf5', save_best_only=True, monitor='loss', mode='min')

reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta=1e-4, mode='min')
callbacks = [mcp_save]


In [153]:
# build the model 

# some old code commented out 
# from tensorflow.python.keras.utils.data_utils import Sequence 
# model.fit(x_batch,y_batch, epochs=1, steps_per_epoch=5, validation_data=validation_data)
-
# model.fit(x_batch,y_batch,
#           epochs=200,batch_size=16, validation_split=0.1, verbose=1)  #this is for the 60 day sequence. 

# history_fert_dayssince_4cov_lr001_rmsprop_lstm_dro20 = model.fit(x_data,y_data,
#          epochs=500,batch_size=16, validation_split=0.1, verbose=1, callbacks = callbacks)



lstm_mod = model.fit(x_data,y_data,sample_weight=sample_weight,
          epochs=500,batch_size=16, validation_split=0.1, verbose=1, callbacks = callbacks)

# is training/building (calibration) and testing (validation) done? 
    # holdout happens in the next script (estimation), but calibration/validation (train/test) should be done at the end of this script...
    

Train on 2165 samples, validate on 241 samples
Epoch 1/500
2165/2165 [==============================] - 2s 930us/step - loss: 0.0040 - val_loss: 7.1417e-04
Epoch 2/500
2165/2165 [==============================] - 1s 508us/step - loss: 0.0029 - val_loss: 6.1161e-04
Epoch 3/500
2165/2165 [==============================] - 1s 491us/step - loss: 0.0026 - val_loss: 6.3285e-04
Epoch 4/500
2165/2165 [==============================] - 1s 498us/step - loss: 0.0025 - val_loss: 5.7838e-04
Epoch 5/500
2165/2165 [==============================] - 1s 490us/step - loss: 0.0024 - val_loss: 6.0200e-04
Epoch 6/500
2165/2165 [==============================] - 1s 496us/step - loss: 0.0024 - val_loss: 5.8236e-04
Epoch 7/500
2165/2165 [==============================] - 1s 491us/step - loss: 0.0024 - val_loss: 6.0273e-04
Epoch 8/500
2165/2165 [==============================] - 1s 505us/step - loss: 0.0023 - val_loss: 5.7848e-04
Epoch 9/500
2165/2165 [==============================] - 1s 505us/step - loss: 0.

2165/2165 [==============================] - 1s 521us/step - loss: 0.0011 - val_loss: 6.5159e-04
Epoch 76/500
2165/2165 [==============================] - 1s 524us/step - loss: 0.0011 - val_loss: 6.5129e-04
Epoch 77/500
2165/2165 [==============================] - 1s 518us/step - loss: 0.0011 - val_loss: 5.9897e-04
Epoch 78/500
2165/2165 [==============================] - 1s 518us/step - loss: 0.0011 - val_loss: 6.0104e-04
Epoch 79/500
2165/2165 [==============================] - 1s 522us/step - loss: 0.0011 - val_loss: 5.9100e-04
Epoch 80/500
2165/2165 [==============================] - 1s 592us/step - loss: 0.0010 - val_loss: 5.9061e-04
Epoch 81/500
2165/2165 [==============================] - 1s 561us/step - loss: 0.0011 - val_loss: 6.0750e-04
Epoch 82/500
2165/2165 [==============================] - 1s 557us/step - loss: 0.0010 - val_loss: 6.2362e-04
Epoch 83/500
2165/2165 [==============================] - 1s 559us/step - loss: 0.0010 - val_loss: 6.6288e-04
Epoch 84/500
2165/2165 

Epoch 147/500
2165/2165 [==============================] - 1s 643us/step - loss: 6.3269e-04 - val_loss: 6.3972e-04
Epoch 148/500
2165/2165 [==============================] - 1s 662us/step - loss: 6.6457e-04 - val_loss: 6.3568e-04
Epoch 149/500
2165/2165 [==============================] - 1s 662us/step - loss: 6.1044e-04 - val_loss: 6.1443e-04
Epoch 150/500
2165/2165 [==============================] - 1s 643us/step - loss: 6.1840e-04 - val_loss: 6.4753e-04
Epoch 151/500
2165/2165 [==============================] - 1s 684us/step - loss: 6.3574e-04 - val_loss: 6.2476e-04
Epoch 152/500
2165/2165 [==============================] - 2s 724us/step - loss: 6.1776e-04 - val_loss: 6.4519e-04
Epoch 153/500
2165/2165 [==============================] - 1s 683us/step - loss: 6.3593e-04 - val_loss: 6.1795e-04
Epoch 154/500
2165/2165 [==============================] - 1s 662us/step - loss: 6.2720e-04 - val_loss: 6.3892e-04
Epoch 155/500
2165/2165 [==============================] - 1s 668us/step - loss:

2165/2165 [==============================] - 2s 698us/step - loss: 4.7308e-04 - val_loss: 6.3211e-04
Epoch 219/500
2165/2165 [==============================] - 1s 664us/step - loss: 4.7459e-04 - val_loss: 6.3808e-04
Epoch 220/500
2165/2165 [==============================] - 1s 692us/step - loss: 4.8556e-04 - val_loss: 6.0319e-04
Epoch 221/500
2165/2165 [==============================] - 1s 683us/step - loss: 4.8481e-04 - val_loss: 6.1098e-04
Epoch 222/500
2165/2165 [==============================] - 1s 681us/step - loss: 4.9692e-04 - val_loss: 6.1257e-04
Epoch 223/500
2165/2165 [==============================] - 1s 672us/step - loss: 4.5799e-04 - val_loss: 6.2462e-04
Epoch 224/500
2165/2165 [==============================] - 1s 674us/step - loss: 4.6472e-04 - val_loss: 6.2430e-04
Epoch 225/500
2165/2165 [==============================] - 1s 670us/step - loss: 4.4832e-04 - val_loss: 6.0877e-04
Epoch 226/500
2165/2165 [==============================] - 1s 670us/step - loss: 4.6459e-04 - 

2165/2165 [==============================] - 1s 660us/step - loss: 3.8890e-04 - val_loss: 6.1149e-04
Epoch 290/500
2165/2165 [==============================] - 1s 659us/step - loss: 3.8309e-04 - val_loss: 6.2703e-04
Epoch 291/500
2165/2165 [==============================] - 1s 669us/step - loss: 3.9157e-04 - val_loss: 6.0909e-04
Epoch 292/500
2165/2165 [==============================] - 1s 655us/step - loss: 4.0354e-04 - val_loss: 6.2632e-04
Epoch 293/500
2165/2165 [==============================] - 2s 703us/step - loss: 3.8301e-04 - val_loss: 6.1631e-04
Epoch 294/500
2165/2165 [==============================] - 1s 660us/step - loss: 3.8587e-04 - val_loss: 6.2274e-04
Epoch 295/500
2165/2165 [==============================] - 1s 660us/step - loss: 3.8748e-04 - val_loss: 6.2668e-04
Epoch 296/500
2165/2165 [==============================] - 1s 686us/step - loss: 3.8478e-04 - val_loss: 6.6931e-04
Epoch 297/500
2165/2165 [==============================] - 1s 660us/step - loss: 3.8576e-04 - 

2165/2165 [==============================] - 1s 662us/step - loss: 3.2976e-04 - val_loss: 5.9983e-04
Epoch 361/500
2165/2165 [==============================] - 1s 661us/step - loss: 3.4533e-04 - val_loss: 5.9994e-04
Epoch 362/500
2165/2165 [==============================] - 1s 661us/step - loss: 3.3996e-04 - val_loss: 6.0060e-04
Epoch 363/500
2165/2165 [==============================] - 1s 663us/step - loss: 3.3726e-04 - val_loss: 6.0729e-04
Epoch 364/500
2165/2165 [==============================] - 1s 662us/step - loss: 3.6175e-04 - val_loss: 6.1603e-04
Epoch 365/500
2165/2165 [==============================] - 1s 664us/step - loss: 3.2499e-04 - val_loss: 6.2839e-04
Epoch 366/500
2165/2165 [==============================] - 1s 667us/step - loss: 3.3308e-04 - val_loss: 6.0198e-04
Epoch 367/500
2165/2165 [==============================] - 1s 669us/step - loss: 3.4015e-04 - val_loss: 6.2386e-04
Epoch 368/500
2165/2165 [==============================] - 1s 678us/step - loss: 3.3058e-04 - 

2165/2165 [==============================] - 1s 665us/step - loss: 2.9568e-04 - val_loss: 5.7167e-04
Epoch 432/500
2165/2165 [==============================] - 1s 679us/step - loss: 3.2349e-04 - val_loss: 6.0374e-04
Epoch 433/500
2165/2165 [==============================] - 1s 671us/step - loss: 3.0141e-04 - val_loss: 5.9052e-04
Epoch 434/500
2165/2165 [==============================] - 1s 670us/step - loss: 3.1073e-04 - val_loss: 5.8636e-04
Epoch 435/500
2165/2165 [==============================] - 2s 708us/step - loss: 3.0485e-04 - val_loss: 6.0525e-04
Epoch 436/500
2165/2165 [==============================] - 1s 672us/step - loss: 3.0425e-04 - val_loss: 5.6200e-04
Epoch 437/500
2165/2165 [==============================] - 1s 675us/step - loss: 3.1192e-04 - val_loss: 6.0799e-04
Epoch 438/500
2165/2165 [==============================] - 2s 703us/step - loss: 3.0440e-04 - val_loss: 5.8473e-04
Epoch 439/500
2165/2165 [==============================] - 1s 664us/step - loss: 2.9755e-04 - 